In [3]:
import pandas as pd
import numpy as np

# Codigo funciones anidado - $Python$

In [1]:
def prestacion_definida(df, fecha_valoracion='31/12/2021', rvs=.015, rvp=.01, h=12, margen_sol=.02,
                        tipo_int=.0022, beta=.15, edad_jubilacion=65):
    import pandas as pd
    import numpy as np
    import planex
    from pandas.core.common import SettingWithCopyWarning
    import warnings
    warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)
    datos = df

    def lx_participes():
        return planex.Prestacion_definida().lx_participes()

    data1 = lx_participes()[0]
    data2 = lx_participes()[1]

    array1 = []
    for i in range(0, len(df['Participe'])):
        array1.append(int(df['Nacimiento'][i].split('/')[2]))

    array2 = []
    for i in range(0, len(df['Participe'])):
        array2.append(int(df['Alta'][i].split('/')[2]))
    r1 = 1 + rvs  ## Revalorizaciòn salarios
    r2 = 1 + rvp  ## Revalorizaciòn pensiones
    interes = (1 + tipo_int)  ##interés
    t1 = np.arange(0, ((120 - edad_jubilacion) * h))  ## mensilidad
    interes3 = interes ** -(t1 / h)  ## fraccionamiento 1/h

    def variation_capital():
        return planex.Prestacion_definida().variation_capital()
    hello = variation_capital()
    ok, ok1 = hello[0], hello[1]

    e1 = round(((pd.to_datetime(datos['Alta'], dayfirst=True) - pd.to_datetime(datos['Nacimiento'], dayfirst=True)) / 365.25) / np.timedelta64(1,
                                                                                                                 'D'))  ## Edad E
    fec_val = []
    for i in range(0, 6):
        fec_val.append(fecha_valoracion)
    fec_val = pd.Series(fec_val)
    act_edad = round(
        ((pd.to_datetime(fec_val, dayfirst=True) - pd.to_datetime(datos['Nacimiento'], dayfirst=True)) / 365.25) / np.timedelta64(1, 'D'))  ## Edad A
    df = df.join(pd.Series(act_edad, name='Edad actuarial'))
    df = df.join(pd.Series(e1, name='Edad entrada'))
    jubi2 = pd.to_numeric(df['Nacimiento'].str.split('/', expand=True)[2]) + edad_jubilacion
    mes = pd.to_numeric(df['Nacimiento'].str.split('/', expand=True)[1])
    df = df.join(pd.Series(jubi2, name='Jubilacion'))
    proyec = round(df['Salario 2021'] * (r1) ** (edad_jubilacion - 1 - df['Edad actuarial']), 2)
    pj = round(beta * proyec / h, 2)
    df['Proyectado'] = proyec
    df['Proyectado'][2] = datos['Salario 2021'][2]

    jubi3 = pd.to_numeric(df['Alta'].str.split('/', expand=True)[2])
    df['PJ corriente'] = pj
    df['PJ corriente'][2] = beta * df['Proyectado'][2] / h
    joan1 = ok['Joan_var'] * (
        (data2.Joan[edad_jubilacion * h + mes[0] + 1:] / data2.Joan[edad_jubilacion * h + mes[0]]).reset_index(
            drop=True))
    pere1 = ok['Pere_var'] * (
        (data2.Pere[edad_jubilacion * h + mes[1] + 1:] / data2.Pere[edad_jubilacion * h + mes[1]]).reset_index(
            drop=True))
    marta1 = ok['Marta_var'] * (
        (data2.Marta[edad_jubilacion * h + mes[2] + 1:] / data2.Marta[edad_jubilacion * h + mes[2]]).reset_index(
            drop=True))
    claudia1 = ok['Claudia_var'] * (
        (data2.Claudia[edad_jubilacion * h + mes[3] + 1:] / data2.Claudia[edad_jubilacion * h + mes[3]]).reset_index(
            drop=True))
    marc1 = ok['Marc_var'] * (
        (data2.Marc[edad_jubilacion * h + mes[4] + 1:] / data2.Marc[edad_jubilacion * h + mes[4]]).reset_index(
            drop=True))
    maria1 = ok['Maria_var'] * (
        (data2.Maria[edad_jubilacion * h + mes[5] + 1:] / data2.Maria[edad_jubilacion * h + mes[5]]).reset_index(
            drop=True))
    new_df = pd.DataFrame({'Joan_VAA': joan1 * interes3, 'Pere_VAA': pere1 * interes3, 'Marta_VAA': marta1 * interes3,
                           'Claudia_VAA': claudia1 * interes3, 'Marc_VAA': marc1 * interes3,
                           'Maria_VAA': maria1 * interes3})
    VAAs = pd.Series(new_df.sum().round(2).to_list(), name='VAAj')

    ## A la fecha actual 2021
    VAA21_joan = round(VAAs[0] * ((1 + tipo_int) ** -(edad_jubilacion - df['Edad actuarial'][0])) * (
            data1['Joan'][edad_jubilacion] / data1['Joan'][df['Edad actuarial'][0]]), 2)
    VAA21_pere = round(VAAs[1] * ((1 + tipo_int) ** -(edad_jubilacion - df['Edad actuarial'][1])) * (
            data1['Pere'][edad_jubilacion] / data1['Pere'][df['Edad actuarial'][1]]), 2)
    VAA21_marta = round(VAAs[2] * ((1 + tipo_int) ** -(edad_jubilacion - df['Edad actuarial'][2])) * (
            data1['Marta'][edad_jubilacion] / data1['Marta'][df['Edad actuarial'][2]]), 2)
    VAA21_claudia = round(VAAs[3] * ((1 + tipo_int) ** -(edad_jubilacion - df['Edad actuarial'][3])) * (
            data1['Claudia'][edad_jubilacion] / data1['Claudia'][df['Edad actuarial'][3]]), 2)
    VAA21_marc = round(VAAs[4] * ((1 + tipo_int) ** -(edad_jubilacion - df['Edad actuarial'][4])) * (
            data1['Marc'][edad_jubilacion] / data1['Marc'][df['Edad actuarial'][4]]), 2)
    VAA21_maria = round(VAAs[5] * ((1 + tipo_int) ** -(edad_jubilacion - df['Edad actuarial'][5])) * (
            data1['Maria'][edad_jubilacion] / data1['Maria'][df['Edad actuarial'][5]]), 2)
    vaas21 = pd.Series([VAA21_joan, VAA21_pere, VAA21_marta, VAA21_claudia, VAA21_marc, VAA21_maria])

    ## A la fecha de alta en la empresa
    VAAp_joan = round(VAAs[0] * ((1 + tipo_int) ** -(jubi2[0] - jubi3[0] + 1)) * (
            data1['Joan'][edad_jubilacion] / data1['Joan'][df['Edad entrada'][0]]), 2)
    VAAp_pere = round(VAAs[1] * ((1 + tipo_int) ** -(jubi2[1] - jubi3[1] + 1)) * (
            data1['Pere'][edad_jubilacion] / data1['Pere'][df['Edad entrada'][1]]), 2)
    VAAp_marta = round(VAAs[2] * ((1 + tipo_int) ** -(jubi2[2] - jubi3[2] + 1)) * (
            data1['Marta'][edad_jubilacion] / data1['Marta'][df['Edad entrada'][2]]), 2)
    VAAp_claudia = round(VAAs[3] * ((1 + tipo_int) ** -(jubi2[3] - jubi3[3] + 1)) * (
            data1['Claudia'][edad_jubilacion] / data1['Claudia'][df['Edad entrada'][3]]), 2)
    VAAp_marc = round(VAAs[4] * ((1 + tipo_int) ** -(jubi2[4] - jubi3[4] + 1)) * (
            data1['Marc'][edad_jubilacion] / data1['Marc'][df['Edad entrada'][4]]), 2)
    VAAp_maria = round(VAAs[5] * ((1 + tipo_int) ** -(jubi2[5] - jubi3[5] + 1)) * (
            data1['Maria'][edad_jubilacion] / data1['Maria'][df['Edad entrada'][5]]), 2)
    vaas_p = pd.Series([VAAp_joan, VAAp_pere, VAAp_marta, VAAp_claudia, VAAp_marc, VAAp_maria])

    ## Coste normal Credito Unitario
    jo1 = vaas21[0] / (edad_jubilacion - df['Edad entrada'][0])
    jo2 = vaas21[1] / (edad_jubilacion - df['Edad entrada'][1])
    jo3 = vaas21[2] / (edad_jubilacion - df['Edad entrada'][2])
    jo4 = vaas21[3] / (edad_jubilacion - df['Edad entrada'][3])
    jo5 = vaas21[4] / (edad_jubilacion - df['Edad entrada'][4])
    jo6 = vaas21[5] / (edad_jubilacion - df['Edad entrada'][5])
    CN_credito = pd.Series([jo1, jo2, jo3, jo4, jo5, jo6])

    ## Valor actual actuarial servicios pasados
    VAASP_CU = pd.Series([CN_credito[0] * (df['Edad actuarial'][0] - df['Edad entrada'][0]),
                          CN_credito[1] * (df['Edad actuarial'][1] - df['Edad entrada'][1]),
                          CN_credito[2] * (df['Edad actuarial'][2] - df['Edad entrada'][2]),
                          CN_credito[3] * (df['Edad actuarial'][3] - df['Edad entrada'][3]),
                          CN_credito[4] * (df['Edad actuarial'][4] - df['Edad entrada'][4]),
                          CN_credito[5] * (df['Edad actuarial'][5] - df['Edad entrada'][5])])
    ## Margen de solvencia
    margen_CU = VAASP_CU * margen_sol

    ## dataframe Credito Unitario
    CU = pd.DataFrame({'VAAj': VAAs, 'VAA 31/12/2021': vaas21,
                       'CN 31/12/2021': CN_credito, 'VAASP 31/12/2021': VAASP_CU, 'MS 31/12/2021': margen_CU}).round(2)
    CU.index = df['Participe'].to_list()
    CU['VAASF 31/12/2021'] = CU.iloc[:, 1] - CU.iloc[:, 3]

    ## Edad de entrada - Coste Normal a la alta
    tiempo1 = np.arange(0, (edad_jubilacion - df['Edad entrada'][0]))
    # jo_1 =
    jo_1 = vaas_p[0] / sum(((1 + rvs) ** tiempo1 * (1 + tipo_int) ** -(tiempo1)) * (
            data1['Joan'][df['Edad entrada'][0] + tiempo1] / data1['Joan'][df['Edad entrada'][0]]))
    tiempo2 = np.arange(0, (edad_jubilacion - df['Edad entrada'][1]))
    # jo_2 =
    jo_2 = vaas_p[1] / sum(((1 + rvs) ** tiempo2 * (1 + tipo_int) ** -(tiempo2)) * (
            data1['Pere'][df['Edad entrada'][1] + tiempo2] / data1['Pere'][df['Edad entrada'][1]]))
    tiempo3 = np.arange(0, (edad_jubilacion - df['Edad entrada'][2]))
    # jo_3 =
    jo_3 = vaas_p[2] / sum(((1 + rvs) ** tiempo3 * (1 + tipo_int) ** -(tiempo3)) * (
            data1['Marta'][df['Edad entrada'][2] + tiempo3] / data1['Claudia'][df['Edad entrada'][2]]))
    tiempo4 = np.arange(0, (edad_jubilacion - df['Edad entrada'][3]))
    # jo_4 =
    jo_4 = vaas_p[3] / sum(((1 + rvs) ** tiempo4 * (1 + tipo_int) ** -(tiempo4)) * (
            data1['Claudia'][df['Edad entrada'][3] + tiempo4] / data1['Marta'][df['Edad entrada'][3]]))
    tiempo5 = np.arange(0, (edad_jubilacion - df['Edad entrada'][4]))
    # jo_5 =
    jo_5 = vaas_p[4] / sum(((1 + rvs) ** tiempo5 * (1 + tipo_int) ** -(tiempo5)) * (
            data1['Marc'][df['Edad entrada'][4] + tiempo5] / data1['Marc'][df['Edad entrada'][4]]))
    tiempo6 = np.arange(0, (edad_jubilacion - df['Edad entrada'][5]))
    # jo_6 =
    jo_6 = vaas_p[5] / sum(((1 + rvs) ** tiempo6 * (1 + tipo_int) ** -(tiempo6)) * (
            data1['Maria'][df['Edad entrada'][5] + tiempo6] / data1['Maria'][df['Edad entrada'][5]]))
    CN_EE = pd.Series([jo_1, jo_2, jo_3, jo_4, jo_5, jo_6]).round(2)  ## Coste Normal Edad de entrada en el 0

    # Coste Normal en el 2021 de los participes
    p1 = CN_EE[0] * (1 + rvs) ** (df['Edad actuarial'][0] - df['Edad entrada'][0])
    p2 = CN_EE[1] * (1 + rvs) ** (df['Edad actuarial'][1] - df['Edad entrada'][1])
    p3 = CN_EE[2] * (1 + rvs) ** (df['Edad actuarial'][2] - df['Edad entrada'][2])
    p4 = CN_EE[3] * (1 + rvs) ** (df['Edad actuarial'][3] - df['Edad entrada'][3])
    p5 = CN_EE[4] * (1 + rvs) ** (df['Edad actuarial'][4] - df['Edad entrada'][4])
    p6 = CN_EE[5] * (1 + rvs) ** (df['Edad actuarial'][5] - df['Edad entrada'][5])

    CN_EE21 = pd.Series([p1, p2, p3, p4, p5, p6]).round(2)

    ## Flujo a Futuro
    def flujo():
        marta_flu = planex.Prestacion_definida().variation_capital()[0]['Marta_var'].shift(2).fillna(0)[:301]
        joan_flu = planex.Prestacion_definida().variation_capital()[0]['Joan_var'].shift(198).fillna(0)[:301]
        flux = pd.DataFrame({'Joan':joan_flu, 'Pere':np.zeros(301),'Marta':marta_flu, 
                        'Claudia':np.zeros(301), 'Marc':np.zeros(301), 
                        'Maria':np.zeros(301)})
        flux['dates'] = pd.date_range(start='2021-12-31', end='2046-12-31', freq='M')
        flux = flux.set_index('dates')
        flux.iloc[-1:, 1] = 672.62
        return flux

    # Valor actual actuarial servicios pasados edad de entradad al 2021
    w1 = np.arange(0, (edad_jubilacion - df['Edad actuarial'][0]))
    joa1 = round(sum(CN_EE21[0] * (1 + rvs) ** w1 * (1 + tipo_int) ** -w1 * (
            data1['Joan'][df['Edad entrada'][0]+(df['Edad actuarial'][0] - df['Edad entrada'][0]) + w1] / data1['Joan'][df['Edad entrada'][0]+(df['Edad actuarial'][0] - df['Edad entrada'][0])])), 2)

    w2 = np.arange(0, (edad_jubilacion - df['Edad actuarial'][1]))
    joa2 = round(sum(CN_EE21[1] * (1 + rvs) ** w2 * (1 + tipo_int) ** -w2 * (
            data1['Pere'][df['Edad entrada'][1]+(df['Edad actuarial'][1] - df['Edad entrada'][1]) + w2] / data1['Pere'][df['Edad actuarial'][1]+(df['Edad actuarial'][1] - df['Edad entrada'][1])])), 2)

    w3 = np.arange(0, (edad_jubilacion - df['Edad actuarial'][2]))
    joa3 = round(sum(CN_EE21[2] * (1 + rvs) ** w3 * (1 + tipo_int) ** -w3 * (
            data1['Marta'][df['Edad entrada'][2]+(df['Edad actuarial'][2] - df['Edad entrada'][2]) + w3] / data1['Marta'][df['Edad entrada'][2]+(df['Edad actuarial'][2] - df['Edad entrada'][2])])), 2)

    w4 = np.arange(0, (edad_jubilacion - df['Edad actuarial'][3]))
    joa4 = round(sum(CN_EE21[3] * (1 + rvs) ** w4 * (1 + tipo_int) ** -w4 * (
            data1['Claudia'][df['Edad entrada'][3]+(df['Edad actuarial'][3] - df['Edad entrada'][3]) + w4] / data1['Claudia'][df['Edad entrada'][3]+(df['Edad actuarial'][3] - df['Edad entrada'][3])])), 2)

    w5 = np.arange(0, (edad_jubilacion - df['Edad actuarial'][4]))
    joa5 = round(sum(CN_EE21[4] * (1 + rvs) ** w5 * (1 + tipo_int) ** -w5 * (
            data1['Marc'][df['Edad entrada'][4]+(df['Edad actuarial'][4] - df['Edad entrada'][4]) + w5] / data1['Marc'][df['Edad entrada'][4]+(df['Edad actuarial'][4] - df['Edad entrada'][4])])), 2)

    w6 = np.arange(0, (edad_jubilacion - df['Edad actuarial'][5]))
    joa6 = round(sum(CN_EE21[5] * (1 + rvs) ** w6 * (1 + tipo_int) ** -w6 * (
            data1['Maria'][df['Edad entrada'][5]+(df['Edad actuarial'][5] - df['Edad entrada'][5]) + w6] / data1['Maria'][df['Edad actuarial'][5]+(df['Edad actuarial'][5] - df['Edad entrada'][5])])), 2)

    serie_vaasp = vaas21 - pd.Series([joa1, joa2, joa3, joa4, joa5, joa6])

    ## Margen metodo de entrada
    mar_EE = serie_vaasp * margen_sol

    df_EE = pd.DataFrame(
        {'VAAj': VAAs, 'VAA 31/12/2021': vaas21, 'CN 31/12/2021': CN_EE21, 'VAASP 31/12/2021': serie_vaasp,
         'VAA en ALTA': vaas_p,
         'MS 31/12/2021': mar_EE.round(2)})
    df_EE['VAASF 31/12/2021'] = df_EE.iloc[:, 1] - df_EE.iloc[:, 3]
    df_EE.index = df['Participe'].to_list()
    print('=' * 100)
    print('Hipotesis')
    print(f'Interés técnico valoración: {tipo_int * 100}%')
    print(f'Revalorizaciòn Salario: {rvs * 100}%')
    print(f'Revalorización Pensión: {rvp * 100}%')
    print(f'Beta Pension: {beta}')
    print('=' * 100)
    print(df.set_index('Participe'))
    # print('=' * 75)
    # print('METODO: Credito Unitario Proyectado')
    # print(round(CU.T, 2))
    # print(CU)
    # print('=' * 75)
    # print('METODO: Edad de entrada')
    # print(df_EE)

    return df_EE, CU, CN_EE, flujo()


In [26]:
def lx_participantes(salarioP_inicial: float = 29944,
                        tipo_salario: float = .0125, rendimiento_fondo: float = .018,
                        aportacion_inicial: float = 368.77 , fondo_c: float = 1364.68,
                        crecimiento_sala: float = .015, 
                        tabla='perm1', jubilacion: int = 65, capital: float = 1, x=48,m=0,
                        n: int = 120, h: int = 12, birth_year: int = 1973, sexo = 'Hombre', 
                        interes: float = .0125, base_tecnica:float = .0175, 
                        fondo_capitalizacion: float = 3486.75):
    import pandas as pd
    import numpy as np
    if tabla == 'perm1':
        f = 'https://raw.githubusercontent.com/Joevalencia/telma/main/telma/PER2020_Col_1er.orden.csv'
        data = pd.read_csv(f)
    elif tabla == 'perm2':
        f = 'https://raw.githubusercontent.com/Joevalencia/telma/main/telma/PER2020_Ind_2ndo.orden.csv'
        data = pd.read_csv(f)
    else:
        print('Solo se permiten tabla perm1 o tabla perm2')
    datatable = data
    frac_freq = h
    if n == 120:
        n = 120 - (x + m)
    elif n != 120:
        n = n
    else:
        print('Error! Type a new "n" ')
    L = np.repeat(1.0, 121)
    L[0] = 1000000
    fem = datatable.iloc[:, 1] * np.exp(-datatable.iloc[:, 2] * (birth_year + np.arange(0, 121) - 2012))
    macho = datatable.iloc[:, 3] * np.exp(-datatable.iloc[:, 4] * (birth_year + np.arange(0, 121) - 2012))
    tabla = .5 * (fem + macho)
    if sexo == 'Hombre':
        for i in range(0, 120):
            L[i + 1] = L[i] * (1 - (macho[i] / 1000))
    elif sexo == 'Mujer':
        
        for i in range(0, 120):
            L[i + 1] = L[i] * (1 - (fem[i] / 1000))
    elif sexo == 'unisex':
        for i in range(0, 120):
            L[i + 1] = L[i] * (1 - (tabla[i] / 1000))
    else:
        
        print('Error. Ingresar "Hombre", "Mujer" o "unisex"')
    L = np.round(L, 4)   # Tablas perm1 anuales
    
    fr = np.arange(frac_freq) / frac_freq
    jon = np.outer(L[1:], fr)
    jon1 = np.outer(L[:-1], (1 - fr))
    jj = (jon + jon1).flatten().round(1)  ## fraccionamiento
    px = jj[np.arange((jubilacion+1)*h, 120*h)]/jj[jubilacion*h] # px desde la jubilacion
    
    
    return L, px, data1.head(10), data2.head(10) ## lx de los participes y lx fraccionada


In [27]:
b = planex.Prestacion_definina().lx_participantes()
b

(array([1.00000000e+06, 9.90581424e+05, 9.90013252e+05, 9.89542248e+05,
        9.89154005e+05, 9.88835434e+05, 9.88571178e+05, 9.88347410e+05,
        9.88148505e+05, 9.87966683e+05, 9.87792022e+05, 9.87615708e+05,
        9.87446338e+05, 9.87267892e+05, 9.87073786e+05, 9.86853426e+05,
        9.86602211e+05, 9.86316221e+05, 9.85994311e+05, 9.85633594e+05,
        9.85237655e+05, 9.84813664e+05, 9.84369953e+05, 9.83917530e+05,
        9.83462837e+05, 9.83013236e+05, 9.82572144e+05, 9.82139587e+05,
        9.81714147e+05, 9.81295936e+05, 9.80881083e+05, 9.80467439e+05,
        9.80053121e+05, 9.79635322e+05, 9.79212721e+05, 9.78783119e+05,
        9.78342554e+05, 9.77887598e+05, 9.77413398e+05, 9.76913944e+05,
        9.76381418e+05, 9.75809094e+05, 9.75189620e+05, 9.74515894e+05,
        9.73779274e+05, 9.72972840e+05, 9.72087294e+05, 9.71113595e+05,
        9.70041187e+05, 9.68858681e+05, 9.67553216e+05, 9.66114101e+05,
        9.64530931e+05, 9.62796321e+05, 9.60905798e+05, 9.588588

###### Ejempo de lo que contienen la sub-funciones

In [50]:
data1.Antoni[65]

929171.0

In [16]:
data2.Antoni[65*12]

929171.0

In [4]:
px = (data2.iloc[65*12+1:]/data2.iloc[65*12]).reset_index()  ## Matriz de px mensuales Participes
px

,index,Antoni,Pere,Marta,Claudia,Marc,Maria
0,781,0.999621,0.999683,0.999737,0.999876,0.999740,0.999893
1,782,0.999242,0.999365,0.999474,0.999753,0.999480,0.999785
2,783,0.998863,0.999048,0.999212,0.999629,0.999220,0.999678
3,784,0.998485,0.998731,0.998949,0.999505,0.998961,0.999571
4,785,0.998106,0.998414,0.998686,0.999382,0.998701,0.999463
...,...,...,...,...,...,...,...
655,1436,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
656,1437,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
657,1438,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
658,1439,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [61]:
px.head()

,index,Antoni,Pere,Marta,Claudia,Marc,Maria
0,781,0.999621,0.999683,0.999737,0.999876,0.999740,0.999893
1,782,0.999242,0.999365,0.999474,0.999753,0.999480,0.999785
2,783,0.998863,0.999048,0.999212,0.999629,0.999220,0.999678
3,784,0.998485,0.998731,0.998949,0.999505,0.998961,0.999571
4,785,0.998106,0.998414,0.998686,0.999382,0.998701,0.999463


In [30]:
def variation_capital():
    import numpy as np
    import pandas as pd
    var = np.roll(np.round(467.96*1.01**np.floor(np.arange(0,673) / 12 ), 2),-6)
    var[-6:] = var[-7]
    var[0] = 0
    
    return var.head(20),ok#, ok1
variation_capital()

(0       0.00
 1     467.96
 2     467.96
 3     467.96
 4     467.96
 5     467.96
 6     472.64
 7     472.64
 8     472.64
 9     472.64
 10    472.64
 11    472.64
 12    472.64
 13    472.64
 14    472.64
 15    472.64
 16    472.64
 17    472.64
 18    477.37
 19    477.37
 dtype: float64,
      Unnamed: 0  Joan_var  Pere_var  Marta_var  Claudia_var  Marc_var  \
 0             0      0.00      0.00       0.00         0.00      0.00   
 1             1    467.96    671.62     752.15       729.40    975.68   
 2             2    467.96    671.62     752.15       740.34    975.68   
 3             3    467.96    671.62     752.15       740.34    975.68   
 4             4    467.96    671.62     752.15       740.34    975.68   
 ..          ...       ...       ...        ...          ...       ...   
 655         655   1045.62   1478.50    1680.62      1629.79   2180.09   
 656         656   1045.62   1478.50    1680.62      1629.79   2180.09   
 657         657   1045.62   1478.50 

In [32]:
d = variation_capital()
d[1]

,Unnamed: 0,Joan_var,Pere_var,Marta_var,Claudia_var,Marc_var,Maria_var
0,0,0.00,0.00,0.00,0.00,0.00,0.00
1,1,467.96,671.62,752.15,729.40,975.68,783.82
2,2,467.96,671.62,752.15,740.34,975.68,783.82
3,3,467.96,671.62,752.15,740.34,975.68,783.82
4,4,467.96,671.62,752.15,740.34,975.68,795.58
...,...,...,...,...,...,...,...
655,655,1045.62,1478.50,1680.62,1629.79,2180.09,1751.39
656,656,1045.62,1478.50,1680.62,1629.79,2180.09,1751.39
657,657,1045.62,1478.50,1680.62,1629.79,2180.09,1751.39
658,658,1045.62,1478.50,1680.62,1629.79,2180.09,1751.39
